# Load libraries

In [ ]:
# !pip install --upgrade torch
# !pip install --upgrade tensorflow
# !pip install --upgrade jax
# !pip install --upgrade keras-nlp
# !pip install --upgrade keras-cv
# !pip install --upgrade keras

In [ ]:
# %config InlineBackend.figure_format = 'retina'

from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [5, 5]
import os
import numpy as np
from skimage import util
from PIL import Image
import math
from skimage.metrics import structural_similarity as ssim
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import load_model
import pandas as pd
import requests
from io import BytesIO
from toloboy.toloboy import RGB2LAB, from_LAB_to_RGB_img
from tqdm import tqdm
tf.experimental.numpy.experimental_enable_numpy_behavior()
from skimage.color import deltaE_ciede2000
import cv2

2024-07-23 11:41:01.194632: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
print(tf.__version__)

2.16.2


# Define helper functions/classes

## Metrics functions

In [ ]:

# def mse(imageA, imageB, nband = 3):
# 	# the 'Mean Squared Error' between the two images is the
# 	# sum of the squared difference between the two images;
# 	# NOTE: the two images must have the same dimension
# 	err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
# 	err /= float(imageA.shape[0] * imageA.shape[1] * nband)
	
# 	# return the MSE, the lower the error, the more "similar"
# 	# the two images are
# 	return err


In [ ]:
# def rmse(imageA, imageB, nband):
# 	# the 'Root Mean Squared Error' between the two images is the
# 	# sum of the squared difference between the two images;
# 	# NOTE: the two images must have the same dimension
# 	err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
# 	err /= float(imageA.shape[0] * imageA.shape[1] * nband)
# 	err = np.sqrt(err)
# 	return err

In [ ]:

def mae(imageA, imageB, bands):
	# the 'Mean Absolute Error' between the two images is the
	# sum of the squared difference between the two images;
	# NOTE: the two images must have the same dimension
	err = np.sum(np.abs(imageA.astype("float") - imageB.astype("float")))
	err /= float(imageA.shape[0] * imageA.shape[1] * bands)
	return err
        

In [ ]:

def psnr(img1, img2):
    # Compute The peak signal-to-noise ratio (PSNR)
    # Higher PSNR values indicate a higher quality of the predicted image.
    mse = np.mean( (img1.astype("float") - img2.astype("float")) ** 2 )
    # print(mse)
    if mse == 0:
        return 100
    PIXEL_MAX = 255.0
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))


In [ ]:
def compute_delta_e_cie2000(img0_rgb,imag1_RGB,Kl=1, KC=1, KH=1):
    
    Lab1 = cv2.cvtColor(img0_rgb, cv2.COLOR_BGR2Lab)
    Lab2 = cv2.cvtColor(imag1_RGB, cv2.COLOR_BGR2Lab)
    L1, a1, b1 = cv2.split(Lab1)
    L2, a2, b2 = cv2.split(Lab2)
    
    
    delta=deltaE_ciede2000(L1,L2, Kl, KC, KH)
    #print(len(delta))
    
    return np.mean(delta)

# Define custom Dataset class


In [ ]:
class SwisstopoDataset:
    def __init__(self, img_indx, transform=None, large_dataset=False, return_label=True, batch_size=32, shuffle=False):
        self.img_indx = img_indx
        self.transform = transform
        self.large_dataset = large_dataset
        self.return_label = return_label
        self.batch_size = batch_size
        self.shuffle = shuffle

        # Set the appropriate port based on the dataset size
        self.port = 1986 if self.large_dataset else 1985

        # Load metadata
        self.metadata_file = self._load_metadata()

    def _load_metadata(self):
        raw_data_csv_file_link = f"https://perritos.myasustor.com:{self.port}/metadata.csv"
        return pd.read_csv(raw_data_csv_file_link, index_col=0)

    def _fetch_image(self, img_id):
        img_in_server_link = f"https://perritos.myasustor.com:{self.port}/data/img_id_{img_id}.jpg"
        response = requests.get(img_in_server_link)
        image = Image.open(BytesIO(response.content))
        return image

    def _process_image(self, img_id):
        image = self._fetch_image(img_id)
        if self.transform is not None:
            image = self.transform(image)
        else:
            image = tf.keras.preprocessing.image.img_to_array(image)
            image = image / 255.0  # Default normalization
        return image

    def _get_label(self, idx):
        return self.metadata_file["class"].iloc[idx]

    def _generator(self):
        if self.shuffle:
            img_indices = np.random.permutation(len(self.img_indx))
        else:
            img_indices = self.img_indx

        for idx in range(len(self.img_indx)):
            image = self._process_image(self.img_indx[idx])
            L, AB = image  # Unpack the transformed image
            if self.return_label:
                label = self._get_label(idx)
                yield (L, AB), label
            else:
                yield L, AB

    def get_dataset(self):
        # Dynamically infer the shapes of L and AB channels
        def _dynamic_output_signature():
            example_image = self._fetch_image(self.img_indx[0])
            example_transformed = self.transform(example_image)
            L, AB = example_transformed
            L_shape = tf.TensorSpec(shape=L.shape, dtype=tf.float32)
            AB_shape = tf.TensorSpec(shape=AB.shape, dtype=tf.float32)
            if self.return_label:
                return ((L_shape, AB_shape), tf.TensorSpec(shape=(), dtype=tf.int64))
            else:
                return (L_shape, AB_shape)

        output_signature = _dynamic_output_signature()

        dataset = tf.data.Dataset.from_generator(self._generator, output_signature=output_signature)
        dataset = dataset.batch(self.batch_size, drop_remainder=True) # use drop reminder to have same size always
        return dataset

# Define transforms


In [ ]:

class LTransformation(object):
    def __init__(self, contrast_range=(0.9, 1), brightness_range=(-0.05, 0.20), noise_var_range=(0, 0.005)):
        self.contrast_range = contrast_range
        self.brightness_range = brightness_range
        self.noise_var_range = noise_var_range

    def _apply_factor(self, L_channel, contrast_factor, brightness_factor):
        # Apply adjusted brightness and contrast to the L channel
        L_adjusted = contrast_factor * L_channel + brightness_factor

        # Clip adjusted L channel to [0, 1]
        L_adjusted = np.clip(L_adjusted, 0, 1)

        return L_adjusted

    def _apply_noise(self, L_channel, noise_var):
        # Apply Gaussian noise to the L channel
        L_noisy = util.random_noise(L_channel, mode='gaussian', var=noise_var)

        # Clip noisy L channel to [0, 1]
        L_noisy = np.clip(L_noisy, 0, 1)

        return L_noisy

    def _randomize_factors(self):
        return np.random.uniform(*self.brightness_range), np.random.uniform(*self.contrast_range)

    def _randomize_noise_var(self):
        return np.random.uniform(*self.noise_var_range)

    def __call__(self, L_channel):
        while True:
            brightness_factor, contrast_factor = self._randomize_factors()
            noise_var = self._randomize_noise_var()

            # Apply adjusted brightness and contrast to the L channel
            L_adjusted = self._apply_factor(L_channel, contrast_factor, brightness_factor)

            # Apply Gaussian noise to the L channel
            L_augmented = self._apply_noise(L_adjusted, noise_var)

            # Check if values are within range
            if 0 <= np.min(L_augmented) <= np.max(L_augmented) <= 1:
                break

        return L_augmented, contrast_factor, brightness_factor, noise_var

def convert_RGB_to_feed_model(img):
    img = np.asarray(img)
    sz_x = img.shape[0]
    sz_y = img.shape[1]

    train_imgs = np.zeros((sz_x, sz_y, 2))
    train_input = np.zeros((sz_x, sz_y, 1))

    R1 = np.reshape(img[:, :, 0], (sz_x * sz_y, 1))
    G1 = np.reshape(img[:, :, 1], (sz_x * sz_y, 1))
    B1 = np.reshape(img[:, :, 2], (sz_x * sz_y, 1))
    L, A, B = RGB2LAB(R1, G1, B1)

    train_input[:, :, 0] = L.reshape((sz_x, sz_y))
    train_imgs[:, :, 0] = np.reshape(A, (sz_x, sz_y))
    train_imgs[:, :, 1] = np.reshape(B, (sz_x, sz_y))

    return train_input, train_imgs


def convert_RGB__and_augment_to_feed_model(img):
    img = np.asarray(img)
    sz_x = img.shape[0]
    sz_y = img.shape[1]

    train_imgs = np.zeros((sz_x, sz_y, 2))
    train_input = np.zeros((sz_x, sz_y, 1))

    R1 = np.reshape(img[:, :, 0], (sz_x * sz_y, 1))
    G1 = np.reshape(img[:, :, 1], (sz_x * sz_y, 1))
    B1 = np.reshape(img[:, :, 2], (sz_x * sz_y, 1))
    L, A, B = RGB2LAB(R1, G1, B1)

    # Apply LTransformation to the L channel
    L_transformation = LTransformation()
    L_augmented, _, _, _ = L_transformation(L.reshape((sz_x, sz_y)))

    train_input[:, :, 0] = L_augmented
    train_imgs[:, :, 0] = np.reshape(A, (sz_x, sz_y))
    train_imgs[:, :, 1] = np.reshape(B, (sz_x, sz_y))

    return train_input, train_imgs

In [ ]:
def convert_to_LAB_transform(image):
    L, AB = convert_RGB_to_feed_model(image)
    return (L, AB)

In [ ]:
def convert_to_LAB_and_augment_transform(image):
    if np.random.rand() < 0.25:  # only apply augmentation to 25% of the data
        L, AB = convert_RGB__and_augment_to_feed_model(image)
    else:
        L, AB = convert_RGB_to_feed_model(image)
    return (L, AB)

# Check info from the images

The data was initially created using the scripts `retrieve_data.ipynb` and stored in a private server for later (re)use.
In the metadata.csv file we get the information on original link, class and coordinates of each image.

NOTE: the following are links stored in a private server, jet they are still publically available.

In [ ]:
is_large_dataset = True

if is_large_dataset:
    server_port = 1986 # Large dataset of ~10K images
else:
    server_port = 1985 # Large dataset of ~10K images
# server_port = 1985 # initial dataset of 3.6K images

raw_data_csv_file_link = f"https://perritos.myasustor.com:{server_port}/metadata.csv"


metadata_raw_df = pd.read_csv(raw_data_csv_file_link, index_col=0)
metadata_raw_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10008 entries, 0 to 10007
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   img_id      10008 non-null  int64  
 1   img_name    10008 non-null  object 
 2   latitude    10008 non-null  float64
 3   longitude   10008 non-null  float64
 4   zoom_level  10008 non-null  int64  
 5   class       10008 non-null  int64  
 6   link        10008 non-null  object 
dtypes: float64(2), int64(3), object(2)
memory usage: 625.5+ KB


# Split the Train, Valid and Test subsets.

We use the column `image_id` from the metadata as index of the images and then we perform standard shufling and splitting.

The final ratio for the train, validation and test dastasets are: 70, 29 and 1 % respectively

In [ ]:
dataX, dataY = metadata_raw_df["img_id"].to_list(), metadata_raw_df["class"] .to_list()

rand_state = 9898
train_ratio = 0.70
validation_ratio = 0.29
test_ratio = 0.01



# train is now 75% of the entire data set
x_train, x_test, y_train, y_test = train_test_split(dataX, dataY, test_size=1 - train_ratio, stratify = dataY, random_state=rand_state)

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio), stratify=y_test, random_state=rand_state)

print(f"the size fo the train dataset is: {len(x_train)}.\nthe size fo the validation dataset is: {len(x_val)}.\nthe size fo the test dataset is: {len(x_test)}.")

the size fo the train dataset is: 7005.
the size fo the validation dataset is: 2902.
the size fo the test dataset is: 101.


In [ ]:
b_size = 1

# Instantiate the dataset
# img_indices = [0, 1, 2, 3, 4, 5]  # Example indices



test_dataset_loader = SwisstopoDataset(x_test,
                           transform=convert_to_LAB_transform,
                           large_dataset=True,
                           return_label=True,
                           batch_size=b_size,
                           shuffle=False)

# train_dataset = train_dataset_loader.get_dataset()
test_dataset = test_dataset_loader.get_dataset()
# valid_dataset = valid_dataset_loader.get_dataset()


In [ ]:
# Get the tf.data.Dataset
# Iterate over the dataset
# for batch in test_dataset:
#     # (L_channel, AB_channels), labels = batch # print with labels
#     # print(L_channel.shape, AB_channels.shape, print(labels.shape))
#     L_channel, AB_channels= batch # print without labels
#     print(L_channel.shape, AB_channels.shape)
#     break

# Load Models

Additional info about the base *Hyper-U-Net* model can be found at the following sources:

- link to [original paper](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9604844/)

- link to [repository](https://github.com/3DOM-FBK/Hyper_U_Net?tab=readme-ov-file)

- link to [model]("https://drive.usercontent.google.com/download?id=19DaA9f1HIOW9PmUz11xKw65fCo3X7-Fw&export=download&authuser=0&confirm=t&uuid=8a03b6f8-6f5d-4bc8-a62d-8b0cfc98d2db&at=APZUnTU9WqjmYlQcAGh22O2M8wXI%3A1717452655512")

NOTE: This will download a multiples large files to your device in the current directory.

Check if model is in the current directory otherwise download it.

In [ ]:
models_sources = {
    "model_name": [
        "base_model",
        # "HyperUnet_retrained_30e",
        # "HyperUnet_retrain_augmented_30e",
        # "HyperUnet_retrain_augmented_noise_25e",
        "HyperUnet_retrain_augmented_noise_corrected_Adam",
        "HyperUnet_retrain_augmented_noise_corrected_rmsprop",
        "HyperUnet_retrain_no_augmented_corrected_Adam",
          ],
    "url": [
        "https://drive.usercontent.google.com/download?id=19DaA9f1HIOW9PmUz11xKw65fCo3X7-Fw&export=download&authuser=0&confirm=t&uuid=8a03b6f8-6f5d-4bc8-a62d-8b0cfc98d2db&at=APZUnTU9WqjmYlQcAGh22O2M8wXI%3A1717452655512",
        # "https://perritos.myasustor.com:1986/Models/HyperUnet_retrained_30e/HyperUnet_retrain1.keras",
        # "https://perritos.myasustor.com:1986/Models/HyperUnet_retrain_augmented/HyperUnet_retrain_augmented_epoch30_valloss0.0011.keras",
        # "https://perritos.myasustor.com:1986/Models/HyperUnet_retrain_augmented_noise/HyperUnet_retrain_augmented_noise_ckpt_epoch25_valloss0.0010.keras",
        "https://perritos.myasustor.com:1986/Models/HyperUnet_retrain_augmented_noise_corrected_Adam/HyperUnet_retrain_augmented_noise_corrected_Adam_ckpt_epoch30_valloss0.0249.keras",
        "https://perritos.myasustor.com:1986/Models/HyperUnet_retrain_augmented_noise_corrected_rmsprop/HyperUnet_retrain_augmented_noise_corrected_rmsprop_ckpt_epoch30_valloss0.0248.keras",
        "https://perritos.myasustor.com:1986/Models/HyperUnet_retrain_no_augmented_corrected_Adam/HyperUnet_retrain_no_augmented_corrected_Adam_ckpt_epoch30_valloss0.0243.keras",
        ],
    "extension": [
        "h5",
        # "keras",
        # "keras",
        # "keras",
        "keras",
        "keras",
        "keras",
    ]
}

for i in range(len(models_sources[next(iter(models_sources.keys()))])):
    model_name = models_sources["model_name"][i]
    url = models_sources["url"][i]
    print(f"Model Name: {model_name}\nURL: {url}\n")

Model Name: base_model
URL: https://drive.usercontent.google.com/download?id=19DaA9f1HIOW9PmUz11xKw65fCo3X7-Fw&export=download&authuser=0&confirm=t&uuid=8a03b6f8-6f5d-4bc8-a62d-8b0cfc98d2db&at=APZUnTU9WqjmYlQcAGh22O2M8wXI%3A1717452655512

Model Name: HyperUnet_retrain_augmented_noise_corrected_Adam
URL: https://perritos.myasustor.com:1986/Models/HyperUnet_retrain_augmented_noise_corrected_Adam/HyperUnet_retrain_augmented_noise_corrected_Adam_ckpt_epoch30_valloss0.0249.keras

Model Name: HyperUnet_retrain_augmented_noise_corrected_rmsprop
URL: https://perritos.myasustor.com:1986/Models/HyperUnet_retrain_augmented_noise_corrected_rmsprop/HyperUnet_retrain_augmented_noise_corrected_rmsprop_ckpt_epoch30_valloss0.0248.keras

Model Name: HyperUnet_retrain_no_augmented_corrected_Adam
URL: https://perritos.myasustor.com:1986/Models/HyperUnet_retrain_no_augmented_corrected_Adam/HyperUnet_retrain_no_augmented_corrected_Adam_ckpt_epoch30_valloss0.0243.keras



In [ ]:
for i in range(len(models_sources[next(iter(models_sources.keys()))])):
    file_name = models_sources["model_name"][i] + "." + models_sources["extension"][i]
    if not os.path.exists(os.path.join(os.curdir,file_name)):
        model_url = models_sources['url'][i]
        !wget -O {file_name}  "$model_url"
    else:
        print("########### No model to load, everything is in the current directory ##############")
    # !wget -O {models_sources["model_name"][i] + "." + models_sources["extension"][i]} {models_sources["url"][i]}

########### No model to load, everything is in the current directory ##############
########### No model to load, everything is in the current directory ##############
########### No model to load, everything is in the current directory ##############
########### No model to load, everything is in the current directory ##############


In [ ]:

models = {}

for i in range(len(models_sources[next(iter(models_sources.keys()))])):
    file_name = models_sources["model_name"][i] + "." + models_sources["extension"][i]
    # print(f"{'*'*5} loading '{models_sources['model_name'][i]}' {'*'*5}")
    print(f"{'*'*5} loading '{file_name}' {'*'*5}")
    models[models_sources["model_name"][i]] = load_model(file_name)
    print(f"{'*'*5} done {'*'*5}")

***** loading 'base_model.h5' *****


/Users/rubencito/micromamba/envs/colorization/lib/python3.11/site-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


***** done *****
***** loading 'HyperUnet_retrain_augmented_noise_corrected_Adam.keras' *****
***** done *****
***** loading 'HyperUnet_retrain_augmented_noise_corrected_rmsprop.keras' *****
***** done *****
***** loading 'HyperUnet_retrain_no_augmented_corrected_Adam.keras' *****
***** done *****


# Plot inference and compare results